# Hyperparameter Tuning for Neural Collaborative Filtering (NCF)

This notebook demonstrates the process of **hyperparameter tuning** for optimizing a **Neural Collaborative Filtering (NCF)** model used to predict movie ratings. The objective is to enhance the performance of the NCF model by systematically exploring and selecting the best hyperparameters for key components of the architecture, such as **embedding sizes**, **learning rates**, **batch sizes**, and **number of epochs**. The notebook begins with data preprocessing, including handling missing values, calculating **weighted ratings**, and encoding categorical variables like **user_id** and **movie_id**. After splitting the data into **training** and **test** sets, we perform hyperparameter tuning using methods like **Grid Search** or **Randomized Search** to find the optimal configuration that minimizes **Mean Absolute Error (MAE)** and improves model generalization. The tuned NCF model is then trained and evaluated, and its performance is compared against the baseline to demonstrate the impact of hyperparameter tuning on the model's predictive accurac.


In [19]:
import numpy as np
import pandas as pd
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [21]:
# Load the dataset
import pandas as pd

# Load the dataset
df = pd.read_csv('MoviesData_Processed.csv')

# Show the first few rows to get an overview of the dataset
df.head()

df.columns

Index(['budget', 'genres', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew', 'year', 'weighted_rating'],
      dtype='object')

## Model Training

Same as we saw in model training file, NCF is trained here before moving to hyper parameter tuning step.

In [23]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# ==========================
# Preprocessing & Data Setup
# ==========================

# Drop NaN values in vote counts and averages
df = df.dropna(subset=['vote_count', 'vote_average'])

# Calculate overall mean vote average (C)
C = df['vote_average'].mean()

# Define m as the 90th percentile of vote_count
m = df['vote_count'].quantile(0.90)

# We filter out movies that have a vote_count less than m
qualified = df[df['vote_count'] >= m].copy()

# Function to compute the weighted rating
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m)) * R + (m/(v+m)) * C

# We calculate weighted rating and create a new column
qualified['weighted_rating'] = qualified.apply(weighted_rating, axis=1)

# Sorting movies by weighted rating
qualified = qualified.sort_values('weighted_rating', ascending=False)

# Generate synthetic user_id and select relevant columns
df['user_id'] = np.random.randint(0, 1000, df.shape[0])  # Assign random user IDs

ratings = df[['user_id', 'id', 'weighted_rating']].dropna()
ratings.rename(columns={'id': 'movie_id'}, inplace=True)

# Normalize weighted_rating
ratings['weighted_rating'] = (ratings['weighted_rating'] - ratings['weighted_rating'].min()) / \
                             (ratings['weighted_rating'].max() - ratings['weighted_rating'].min())

# Clip negative values (if any)
ratings['weighted_rating'] = ratings['weighted_rating'].clip(lower=0)

# Encode users and movies
total_users = ratings['user_id'].nunique()
total_movies = ratings['movie_id'].nunique()

user2idx = {user: idx for idx, user in enumerate(ratings['user_id'].unique())}
movie2idx = {movie: idx for idx, movie in enumerate(ratings['movie_id'].unique())}

ratings.loc[:, 'user_id'] = ratings['user_id'].map(user2idx)
ratings.loc[:, 'movie_id'] = ratings['movie_id'].map(movie2idx)

# Train-test split
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

# ==========================
# PyTorch Dataset Class
# ==========================

class MovieDataset(Dataset):
    def __init__(self, data):
        self.users = torch.tensor(data['user_id'].values, dtype=torch.long)
        self.movies = torch.tensor(data['movie_id'].values, dtype=torch.long)
        self.ratings = torch.tensor(data['weighted_rating'].values, dtype=torch.float32)

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.users[idx], self.movies[idx], self.ratings[idx]

# DataLoaders
train_dataset = MovieDataset(train_data)
test_dataset = MovieDataset(test_data)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# ==========================
# Neural Collaborative Filtering (NCF) Model
# ==========================

class NCF(nn.Module):
    def __init__(self, num_users, num_movies, embed_size=64):
        super(NCF, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embed_size)
        self.movie_embedding = nn.Embedding(num_movies, embed_size)
        self.fc_layers = nn.Sequential(
            nn.Linear(embed_size * 2, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )
    
    def forward(self, user, movie):
        user_embedded = self.user_embedding(user)
        movie_embedded = self.movie_embedding(movie)
        interaction = torch.cat([user_embedded, movie_embedded], dim=-1)
        output = self.fc_layers(interaction)
        return output.squeeze()

# Initialize model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ncf_model = NCF(total_users, total_movies).to(device)
criterion = nn.L1Loss()  # Mean Absolute Error
optimizer = optim.AdamW(ncf_model.parameters(), lr=0.001, weight_decay=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# ==========================
# Model Weight Initialization
# ==========================

def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

ncf_model.apply(weights_init)

# ==========================
# Training Loop
# ==========================

def train(model, train_loader, criterion, optimizer, scheduler, epochs=20):
    model.train()
    best_loss = float('inf')
    patience, counter = 3, 0
    for epoch in range(epochs):
        total_loss = 0
        for users, movies, ratings in train_loader:
            users, movies, ratings = users.to(device), movies.to(device), ratings.to(device)
            optimizer.zero_grad()
            predictions = model(users, movies)
            loss = criterion(predictions, ratings)
            loss.backward()

            # Gradient clipping
            for param in model.parameters():
                param.grad.data.clamp_(-1, 1)

            optimizer.step()
            total_loss += loss.item()
        scheduler.step()
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")
        
        if avg_loss < best_loss:
            best_loss = avg_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered")
                break

# ==========================
# Evaluation Function
# ==========================

def evaluate(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    total_absolute_error = 0  # Variable to store the sum of absolute errors
    with torch.no_grad():
        for users, movies, ratings in test_loader:
            users, movies, ratings = users.to(device), movies.to(device), ratings.to(device)
            predictions = model(users, movies)
            
            # Compute the loss
            loss = criterion(predictions, ratings)
            total_loss += loss.item()
            
            # Calculate absolute errors for MAE
            absolute_error = torch.abs(predictions - ratings)
            total_absolute_error += absolute_error.sum().item()
    
    # Calculate MAE and average loss
    mae = total_absolute_error / len(test_loader.dataset)
    avg_loss = total_loss / len(test_loader)
    
    print(f"Test Loss: {avg_loss:.4f}, Test MAE: {mae:.4f}")

# ==========================
# Run Training and Evaluation
# ==========================

train(ncf_model, train_loader, criterion, optimizer, scheduler, epochs=20)
evaluate(ncf_model, test_loader, criterion)

Epoch 1, Loss: 0.4198
Epoch 2, Loss: 0.2443
Epoch 3, Loss: 0.1615
Epoch 4, Loss: 0.1199
Epoch 5, Loss: 0.0980
Epoch 6, Loss: 0.0902
Epoch 7, Loss: 0.0907
Epoch 8, Loss: 0.0891
Epoch 9, Loss: 0.0870
Epoch 10, Loss: 0.0843
Epoch 11, Loss: 0.0865
Epoch 12, Loss: 0.0837
Epoch 13, Loss: 0.0833
Epoch 14, Loss: 0.0832
Epoch 15, Loss: 0.0856
Epoch 16, Loss: 0.0854
Epoch 17, Loss: 0.0856
Early stopping triggered
Test Loss: 0.0778, Test MAE: 0.0828


### Neural Collaborative Filtering (NCF) with Hyperparameter Optimization

This code implements a **Neural Collaborative Filtering (NCF)** model for predicting movie ratings, where the hyperparameters of the model are optimized using **Optuna**. The first part of the code simulates user-item interactions by generating **random user IDs** and **weighted ratings** for the movies. After selecting and normalizing the ratings, user and movie IDs are **encoded** to create unique mappings. The data is then split into **train** and **test** sets, and a **PyTorch Dataset** class is used for data handling. The core model, `NCF`, is defined with **embedding layers** for both users and movies, followed by several **fully connected layers** with **dropout** for regularization. The **objective function** of the hyperparameter optimization process utilizes Optuna to sample various hyperparameters such as embedding size, layer configuration, dropout rate, learning rate, batch size, and weight decay. The model is trained for **10 epochs**, and the performance is evaluated using **mean absolute error (L1 loss)**. Finally, Optuna searches for the **best hyperparameters** by minimizing the test loss over a set of trials, with the best combination of hyperparameters printed at the end.

In [25]:
# Simulate user-item interactions (Assuming a rating-like system based on popularity)
df['user_id'] = np.random.randint(0, 1000, df.shape[0])  
df['weighted_rating'] = np.random.randint(1, 6, df.shape[0])  # This is your custom rating system

# Select relevant columns
ratings = df[['user_id', 'id', 'weighted_rating']]
ratings.rename(columns={'id': 'movie_id'}, inplace=True)

# Normalize ratings to [0, 1]
ratings['weighted_rating'] = (ratings['weighted_rating'] - 1) / 4  

# Encode users and movies
total_users = ratings['user_id'].nunique()
total_movies = ratings['movie_id'].nunique()
user2idx = {user: idx for idx, user in enumerate(ratings['user_id'].unique())}
movie2idx = {movie: idx for idx, movie in enumerate(ratings['movie_id'].unique())}

ratings['user_id'] = ratings['user_id'].map(user2idx)
ratings['movie_id'] = ratings['movie_id'].map(movie2idx)

# Train-test split
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

# PyTorch Dataset class
class MovieDataset(Dataset):
    def __init__(self, data):
        self.users = torch.tensor(data['user_id'].values, dtype=torch.long)
        self.movies = torch.tensor(data['movie_id'].values, dtype=torch.long)
        self.ratings = torch.tensor(data['weighted_rating'].values, dtype=torch.float32)

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.users[idx], self.movies[idx], self.ratings[idx]

# Model with Hyperparameter Optimization
class NCF(nn.Module):
    def __init__(self, num_users, num_movies, embed_size, hidden_layers, dropout):
        super(NCF, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embed_size)
        self.movie_embedding = nn.Embedding(num_movies, embed_size)
        
        layers = []
        input_size = embed_size * 2
        for units in hidden_layers:
            layers.append(nn.Linear(input_size, units))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            input_size = units
        layers.append(nn.Linear(input_size, 1))
        
        self.fc_layers = nn.Sequential(*layers)

    def forward(self, user, movie):
        user_embedded = self.user_embedding(user)
        movie_embedded = self.movie_embedding(movie)
        interaction = torch.cat([user_embedded, movie_embedded], dim=-1)
        output = self.fc_layers(interaction)
        return output.squeeze()

# Hyperparameter Optimization Function
def objective(trial):
    # Sample hyperparameters
    embed_size = trial.suggest_categorical("embed_size", [32, 64, 128])
    hidden_layers = [trial.suggest_int(f"layer_{i}", 32, 256, step=32) for i in range(trial.suggest_int("num_layers", 1, 3))]
    dropout = trial.suggest_float("dropout", 0.1, 0.3)
    lr = trial.suggest_loguniform("lr", 0.0005, 0.005)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-5, 1e-2)

    # DataLoader
    train_loader = DataLoader(MovieDataset(train_data), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(MovieDataset(test_data), batch_size=batch_size, shuffle=False)

    # Model, Loss, Optimizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = NCF(total_users, total_movies, embed_size, hidden_layers, dropout).to(device)
    criterion = nn.L1Loss()  # Mean Absolute Error
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    # Training
    model.train()
    for epoch in range(10):  # Using 10 epochs per trial for quick evaluation
        total_loss = 0
        for users, movies, ratings in train_loader:
            users, movies, ratings = users.to(device), movies.to(device), ratings.to(device)
            optimizer.zero_grad()
            predictions = model(users, movies)
            loss = criterion(predictions, ratings)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
    
    # Evaluation
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for users, movies, ratings in test_loader:
            users, movies, ratings = users.to(device), movies.to(device), ratings.to(device)
            predictions = model(users, movies)
            loss = criterion(predictions, ratings)
            total_loss += loss.item()
    
    test_loss = total_loss / len(test_loader)
    return test_loss  # Optuna minimizes this loss

# Run Optuna Hyperparameter Tuning
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

# Print Best Hyperparameters
print("Best Hyperparameters:", study.best_params)

C:\Users\anagh\AppData\Local\Temp\ipykernel_26048\643855626.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings.rename(columns={'id': 'movie_id'}, inplace=True)
C:\Users\anagh\AppData\Local\Temp\ipykernel_26048\643855626.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['weighted_rating'] = (ratings['weighted_rating'] - 1) / 4
C:\Users\anagh\AppData\Local\Temp\ipykernel_26048\643855626.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

Best Hyperparameters: {'embed_size': 32, 'num_layers': 2, 'layer_0': 160, 'layer_1': 256, 'dropout': 0.2351776451471082, 'lr': 0.0005784837063420347, 'batch_size': 128, 'weight_decay': 0.0006577726351663704}


## Model Retraining with suggested hyperparameters

In [31]:
# ==========================
# Preprocessing & Data Setup
# ==========================

# Assume df is already loaded with movie data
# Drop NaN values in vote counts and averages
df = df.dropna(subset=['vote_count', 'vote_average'])

# Calculate overall mean vote average (C)
C = df['vote_average'].mean()

# Define m as the 90th percentile of vote_count
m = df['vote_count'].quantile(0.90)

# We filter out movies that have a vote_count less than m
qualified = df[df['vote_count'] >= m].copy()

# Function to compute the weighted rating
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m)) * R + (m/(v+m)) * C

# We calculate weighted rating and create a new column
qualified['weighted_rating'] = qualified.apply(weighted_rating, axis=1)

# Sorting movies by weighted rating
qualified = qualified.sort_values('weighted_rating', ascending=False)

# Generate synthetic user_id and select relevant columns
df['user_id'] = np.random.randint(0, 1000, df.shape[0])  # Assign random user IDs

ratings = df[['user_id', 'id', 'weighted_rating']].dropna()
ratings.rename(columns={'id': 'movie_id'}, inplace=True)

# Normalize weighted_rating
ratings['weighted_rating'] = (ratings['weighted_rating'] - ratings['weighted_rating'].min()) / \
                             (ratings['weighted_rating'].max() - ratings['weighted_rating'].min())

# Clip negative values (if any)
ratings['weighted_rating'] = ratings['weighted_rating'].clip(lower=0)

# Encode users and movies
total_users = ratings['user_id'].nunique()
total_movies = ratings['movie_id'].nunique()

user2idx = {user: idx for idx, user in enumerate(ratings['user_id'].unique())}
movie2idx = {movie: idx for idx, movie in enumerate(ratings['movie_id'].unique())}

ratings.loc[:, 'user_id'] = ratings['user_id'].map(user2idx)
ratings.loc[:, 'movie_id'] = ratings['movie_id'].map(movie2idx)

# Train-test split
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

# ==========================
# PyTorch Dataset Class
# ==========================

class MovieDataset(Dataset):
    def __init__(self, data):
        self.users = torch.tensor(data['user_id'].values, dtype=torch.long)
        self.movies = torch.tensor(data['movie_id'].values, dtype=torch.long)
        self.ratings = torch.tensor(data['weighted_rating'].values, dtype=torch.float32)

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.users[idx], self.movies[idx], self.ratings[idx]

# DataLoaders
train_dataset = MovieDataset(train_data)
test_dataset = MovieDataset(test_data)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# ==========================
# Neural Collaborative Filtering (NCF) Model
# ==========================

class NCF(nn.Module):
    def __init__(self, num_users, num_movies, embed_size=32, hidden_layers=[160, 256], dropout=0.2351776451471082):
        super(NCF, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embed_size)
        self.movie_embedding = nn.Embedding(num_movies, embed_size)
        
        layers = []
        input_size = embed_size * 2
        for units in hidden_layers:
            layers.append(nn.Linear(input_size, units))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            input_size = units
        layers.append(nn.Linear(input_size, 1))
        
        self.fc_layers = nn.Sequential(*layers)

    def forward(self, user, movie):
        user_embedded = self.user_embedding(user)
        movie_embedded = self.movie_embedding(movie)
        interaction = torch.cat([user_embedded, movie_embedded], dim=-1)
        output = self.fc_layers(interaction)
        return output.squeeze()

# Initialize model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ncf_model = NCF(total_users, total_movies, embed_size=32, hidden_layers=[160, 256], dropout=0.2351776451471082).to(device)
criterion = nn.L1Loss()  # Mean Absolute Error
optimizer = optim.AdamW(ncf_model.parameters(), lr=0.0005784837063420347, weight_decay=0.0006577726351663704)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# ==========================
# Model Weight Initialization
# ==========================

def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

ncf_model.apply(weights_init)

# ==========================
# Training Loop
# ==========================

def train(model, train_loader, criterion, optimizer, scheduler, epochs=20):
    model.train()
    best_loss = float('inf')
    patience, counter = 3, 0
    for epoch in range(epochs):
        total_loss = 0
        for users, movies, ratings in train_loader:
            users, movies, ratings = users.to(device), movies.to(device), ratings.to(device)
            optimizer.zero_grad()
            predictions = model(users, movies)
            loss = criterion(predictions, ratings)
            loss.backward()

            # Gradient clipping
            for param in model.parameters():
                param.grad.data.clamp_(-1, 1)

            optimizer.step()
            total_loss += loss.item()
        scheduler.step()
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")
        
        if avg_loss < best_loss:
            best_loss = avg_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered")
                break

# ==========================
# Evaluation Function
# ==========================

def evaluate(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    total_absolute_error = 0  # Variable to store the sum of absolute errors
    with torch.no_grad():
        for users, movies, ratings in test_loader:
            users, movies, ratings = users.to(device), movies.to(device), ratings.to(device)
            predictions = model(users, movies)
            
            # Compute the loss
            loss = criterion(predictions, ratings)
            total_loss += loss.item()
            
            # Calculate absolute errors for MAE
            absolute_error = torch.abs(predictions - ratings)
            total_absolute_error += absolute_error.sum().item()
    
    # Calculate MAE and average loss
    mae = total_absolute_error / len(test_loader.dataset)
    avg_loss = total_loss / len(test_loader)
    
    print(f"Test Loss: {avg_loss:.4f}, Test MAE: {mae:.4f}")

# ==========================
# Run Training and Evaluation
# ==========================

train(ncf_model, train_loader, criterion, optimizer, scheduler, epochs=20)
evaluate(ncf_model, test_loader, criterion)

Epoch 1, Loss: 0.5500
Epoch 2, Loss: 0.3830
Epoch 3, Loss: 0.3663
Epoch 4, Loss: 0.3543
Epoch 5, Loss: 0.3427
Epoch 6, Loss: 0.3284
Epoch 7, Loss: 0.3176
Epoch 8, Loss: 0.3130
Epoch 9, Loss: 0.3153
Epoch 10, Loss: 0.3135
Epoch 11, Loss: 0.3059
Epoch 12, Loss: 0.3081
Epoch 13, Loss: 0.3039
Epoch 14, Loss: 0.3010
Epoch 15, Loss: 0.3000
Epoch 16, Loss: 0.3010
Epoch 17, Loss: 0.2932
Epoch 18, Loss: 0.2976
Epoch 19, Loss: 0.2996
Epoch 20, Loss: 0.3000
Early stopping triggered
Test Loss: 0.3167, Test MAE: 0.3173


### Addressing the Scenario: Post-Hyperparameter Tuning Performance Degradation

After performing hyperparameter tuning using **Optuna**, the model results showed that despite a **decrease in training loss** over the epochs, the **test loss** and **Mean Absolute Error (MAE)** increased. This indicates potential **overfitting**, where the model is performing well on the training data but struggles to generalize to unseen data. In this case, despite improvements during training, the model's ability to predict accurately on the test set has diminished. 
**Future Scope**: To address this issue, several potential solutions could be explored, first, **regularization techniques** such as **early stopping** (which was already triggered) could be fine-tuned to avoid overfitting, adjusting the **dropout rate** or exploring **weight decay** more rigorously. Additionally, increasing the **amount of training data**, using a different **model architecture**, or reviewing the **hyperparameters** (such as embedding sizes and learning rates) to ensure they are balanced and not too aggressive could help mitigate this issue. Finally, it would be helpful to conduct **cross-validation** during hyperparameter search to better evaluate the model's generalization capability before finalizing the tuning process.

## Model Saving

In [35]:
# Save the model
torch.save(ncf_model.state_dict(), 'ncf_model.pth')

# Save the optimizer state
torch.save(optimizer.state_dict(), 'optimizer.pth')

print("Model and optimizer saved successfully.")

Model and optimizer saved successfully.
